**📌 Secure Shell (SSH) & SCP File Transfer: Project Overview & Findings**

🔹 Introduction: Understanding Secure Shell (SSH) in Our Setup
Secure Shell (SSH) is a network protocol that allows for secure remote access to another machine over an encrypted connection. In our setup:
The MacBook acts as the SSH server (listens for incoming connections).
The Dell acts as the SSH client (initiates the connection).

**🔑 How SSH Works in This Project**
1️⃣ The Dell (client) sends an SSH request to the MacBook (server), establishing an encrypted connection.
2️⃣ The MacBook responds, initiating an SSH handshake, where both devices agree on:
Supported encryption algorithms
Authentication methods
Session keys for encryption
3️⃣ Once the SSH session is established, the Dell requests a file transfer using SCP (Secure Copy Protocol), which runs over SSH.
4️⃣ The MacBook encrypts and transmits the file securely to the Dell.
5️⃣ Finally, the SSH session is terminated either gracefully (FIN/ACK) or forcefully (RST packet).

**📌 Key Insight:**
SSH ensures all data, including authentication and file transfers, remains encrypted, preventing eavesdropping.
SCP (Secure Copy Protocol) is a simple, secure way to transfer files over SSH.

**🔹 Project Objectives & Execution**
📝 Goals of the Project
✔️ Enable SSH on the MacBook and confirm it is listening for connections.
✔️ Use **Nmap** on the Dell to verify that SSH (port 22) is open and available.
✔️ Capture SSH session traffic in Wireshark to analyze encryption and connection behavior.
✔️ Perform an SCP file transfer from the MacBook to the Dell.
✔️ Investigate the SSH session closure to ensure the connection terminates properly.
✔️ Confirm encryption in Wireshark to verify that the file contents are never visible in plaintext.

**🔎 Key Investigation Steps & Findings**

**🔹 Step 1: Establishing SSH & Verifying Availability**
Enabled SSH on the MacBook (sudo systemsetup -setremotelogin on).
Checked that SSH was running (ps aux | grep sshd).
Used Nmap on the Dell to scan SSH port (nmap -p 22 -sV 192.168.1.148).
Result: OpenSSH 9.7 was detected, confirming the SSH service was active.

**🔹 Step 2: Capturing SSH Traffic in Wireshark**
Applied SSH filter (tcp.port == 22) to isolate SSH traffic.
Began capturing packets on the Dell before initiating SCP transfer.

**🔹 Step 3: Performing SCP File Transfer**
Ran SCP on the Dell to retrieve the file:
scp user@192.168.1.148:/Users/user/Desktop/sample.csv ~/Desktop/
**Expected Behavior:**
SSH authentication occurs.
Encrypted data transfer follows.
The file appears on the Dell’s desktop.
**Actual Behavior:**
File was successfully transferred.
Wireshark captured SSH packet exchange.

**🔹 Step 4: Unexpected Behavior – SSH Closure Investigation**
📌 Problem:
We expected a clean SSH session closure (FIN/ACK sequence), but it was missing.
Wireshark did not show a proper termination.

**📌 Investigation:**
Checked for active SSH sessions on the MacBook (ps aux | grep sshd).
Result: No active SSH sessions.
Checked for active SSH sessions on the Dell (netstat -ano | findstr :22).
Result: No active connections.
Applied Wireshark filter to find RST or FIN packets (tcp.flags.fin==1 || tcp.flags.reset==1).
Result: Found a TCP Reset (RST) from Dell to MacBook.

**📌 Conclusion:**
✔️ The Dell forcefully terminated the SSH session via a TCP Reset (RST) instead of a FIN/ACK sequence.
✔️ This is likely a behavior of SCP or the SSH client, cutting the session abruptly after file transfer completion.

**🔹 Step 5: Confirming SCP Encryption in Wireshark**
Filtering for SSH file transfer packets (tcp.port == 22), we analyzed key frames:

🔹 Frame 4525 (Server → Client)
Length: 682 bytes
Encryption Algorithm: chacha20-poly1305@openssh.com
MAC (Integrity Check): Implicitly included
Conclusion: The payload is fully encrypted.

🔹 Frame 4533 (Server → Client)
Length: 410 bytes
MAC (Authentication Code) Ensures Integrity
Encrypted Payload Confirmed:
Encrypted Packet: 9cc6fa6375d5dd36f5a11e75edb66de2f...
**Conclusion: No plaintext file contents were visible, confirming encryption.**


📌 Final Takeaway:
✔️ SCP fully encrypts file transfers, ensuring confidentiality and integrity.

**✅ Project Summary & Lessons Learned**
🔹 Key Takeaways
✅ SSH ensures secure, encrypted remote connections.
✅ SCP transfers files securely over SSH.
✅ Wireshark helps analyze encrypted communication without exposing file contents.
✅ Unexpected behaviors (e.g., missing FIN/ACK) can be diagnosed with TCP flags.
✅ Filtering for tcp.port == 22 in Wireshark isolates SSH traffic effectively.

**🔹 What We Learned About Troubleshooting SSH**
✔️ Missing FIN/ACK packets don’t necessarily mean an issue—TCP Resets are normal in some cases.
✔️ Using tools like ps aux, netstat, and Wireshark can confirm session status.
✔️ SSH traffic can be confirmed as encrypted by checking the encryption algorithm in Wireshark.

**📌 Final Conclusion**
This project successfully demonstrated how SSH and SCP work together to securely transfer files, how to capture and analyze encrypted SSH traffic in Wireshark, and how to troubleshoot unexpected behaviors like session termination anomalies.

**🚀 Potential Next Steps:**
Explore SFTP (Secure FTP) vs. SCP and analyze differences in Wireshark.
Test how different SSH versions affect encryption algorithms in packet captures.
Experiment with SSH session persistence to observe different closure methods.

**Jupyter Notebook Markdown Cell: SSH Connection Timeout Issue (PRIOR TO ABOVE SUCCESFUL FILE TRANSFER)**

🚨 **Issue Encountered: SSH Connection Timeout (Troubleshooting Case Study)**  
 **🔹 Description of the Issue**
During **Step 6: SCP File Transfer**, the Dell attempted to establish an **SSH connection to the Mac** (`192.168.1.148:22`), but the connection **timed out**, preventing the file transfer.  

 **🔹 Terminal Output on Dell**
ssh: connect to host 192.168.1.148 port 22: Connection timed out
scp: Connection closed
📌 This indicates that the Dell is unable to establish an SSH session with the Mac.

**🔹 Wireshark Packet Capture Analysis**
Frame 3209: TCP SYN packet sent from Dell to Mac (192.168.1.148:22).
No SYN-ACK response from the Mac (indicating that the Mac did not acknowledge the connection request).
Wireshark classified this as "Bad TCP", meaning the connection attempt was incomplete.
TCP "SYN_SENT" status confirms that the Dell sent the request, but no response was received.

**📌 Key Takeaway from Wireshark: The Dell is trying to communicate with the Mac via SSH, but the Mac is not responding.**
 Possible Causes
1️⃣ The Mac’s SSH service may not be running or listening properly.
2️⃣ The Mac’s firewall may be blocking SSH connections.
3️⃣ SSH may be misconfigured, only listening on 127.0.0.1:22 (local-only).
**4️⃣ Network instability (Wi-Fi issues) could be affecting communication.**

🔹 Next Steps: Troubleshooting Plan
✅ Step 1: Check if SSH is running on the Mac
sudo lsof -iTCP -sTCP:LISTEN | grep ssh
If no output → SSH is not running (restart it).
If output shows *.22 LISTEN, SSH is running correctly.

✅ Step 2: Verify Mac's Firewall Settings
sudo /usr/libexec/ApplicationFirewall/socketfilterfw --getglobalstate
If firewall is enabled, try disabling it temporarily:
sudo /usr/libexec/ApplicationFirewall/socketfilterfw --setglobalstate off

✅ Step 3: Confirm SSH Is Listening on the Correct Interface
netstat -an | grep LISTEN | grep 22
If SSH only listens on 127.0.0.1, we must configure it to allow external connections.

✅ Step 4: Test SSH Locally on the Mac
ssh localhost
If this fails, SSH is not functioning properly.

✅ Step 5: Test SSH from the Dell Again
telnet 192.168.1.148 22
If connection is refused, SSH is still unreachable.

**🔹 Lessons Learned**
✅ Wireshark’s TCP flags (SYN_SENT) help diagnose failed SSH connections.
✅ A connection timeout means the remote system is not responding at all—either due to a firewall, misconfiguration, or service failure.
✅ Using terminal commands (lsof, netstat, telnet) helps diagnose SSH issues efficiently.
**Most likely:  Network instability (Wi-Fi issues) could be affecting communication.**
Networking problems like this are common and can be methodically solved using systematic troubleshooting.



**🛠️ **Analyzing Secure Shell (SSH) Traffic in Wireshark**  

 **🔹 Strategy & Approach**
To analyze **SSH session establishment and SCP file transfer**, we focus on key frames that indicate critical steps in the process.  

- **Step 1: Identify the First Packets of the SSH Session**
  - We start by locating the **first** packets of the SSH session.  
  - This tells us when the Dell (client) first communicates with the Mac (server).  
  - We use these packets to confirm that SSH is working **before encryption begins**.  
  - We selected **Frames 4458, 4461, and 4462** because they show the **initial handshake and encryption setup**.  

---

 **📌 Step 1: Understanding the Frames Found**
Each frame represents a **different stage** of SSH session initiation.  

 **🖧 Frame 4458: SSH Initial Communication**
- **Length:** **87 bytes**  
- **Source:** `192.168.1.185 (Dell)` → **Destination:** `192.168.1.148 (Mac)`  
- **Protocol:** SSH  
- **Details:**
  - This is likely the **SSH protocol version exchange**.
  - The Dell (client) is sending an **SSH version announcement** to the Mac (server).
  - **Why it matters:** Confirms that the client is requesting an SSH session.
  Frame 4458: 87 bytes on wire (696 bits), 87 bytes captured (696 bits) on interface \Device\NPF_{EAF66F3F-8F00-47F0-827E-72FB128923A3}, id 0
Ethernet II, Src: e0:ad:47:20:d9:0a (e0:ad:47:20:d9:0a), Dst: Apple_db:fd:32 (74:a6:cd:db:fd:32)
Internet Protocol Version 4, Src: 192.168.1.185, Dst: 192.168.1.148
Transmission Control Protocol, Src Port: 35050, Dst Port: 22, Seq: 1, Ack: 1, Len: 33
    Source Port: 35050
    Destination Port: 22
    [Stream index: 34]
    [Stream Packet Number: 4]
    [Conversation completeness: Complete, WITH_DATA (47)]
    [TCP Segment Len: 33]
    Sequence Number: 1    (relative sequence number)
    Sequence Number (raw): 203388168
    [Next Sequence Number: 34    (relative sequence number)]
    Acknowledgment Number: 1    (relative ack number)
    Acknowledgment number (raw): 399429055
    0101 .... = Header Length: 20 bytes (5)
    Flags: 0x018 (PSH, ACK)
    Window: 256
    [Calculated window size: 65536]
    [Window size scaling factor: 256]
    Checksum: 0xf23b [unverified]
    [Checksum Status: Unverified]
    Urgent Pointer: 0
    [Timestamps]
    [SEQ/ACK analysis]
    TCP payload (33 bytes)
SSH Protocol
    Protocol: SSH-2.0-OpenSSH_for_Windows_9.5
    [Direction: client-to-server]
  

---

 **🖧 Frame 4461: SSH Server Response**
- **Length:** **75 bytes**  
- **Source:** `192.168.1.148 (Mac)` → **Destination:** `192.168.1.185 (Dell)`  
- **Protocol:** SSH  
- **Details:**
  - The Mac (server) is responding to the Dell’s SSH request.
  - This likely includes the **server’s protocol version**.
  - **Why it matters:** Confirms that the SSH daemon on the Mac is actively responding to requests. 
  Frame 4461: 75 bytes on wire (600 bits), 75 bytes captured (600 bits) on interface \Device\NPF_{EAF66F3F-8F00-47F0-827E-72FB128923A3}, id 0
Ethernet II, Src: Apple_db:fd:32 (74:a6:cd:db:fd:32), Dst: e0:ad:47:20:d9:0a (e0:ad:47:20:d9:0a)
Internet Protocol Version 4, Src: 192.168.1.148, Dst: 192.168.1.185
Transmission Control Protocol, Src Port: 22, Dst Port: 35050, Seq: 1, Ack: 34, Len: 21
    Source Port: 22
    Destination Port: 35050
    [Stream index: 34]
    [Stream Packet Number: 7]
    [Conversation completeness: Complete, WITH_DATA (47)]
    [TCP Segment Len: 21]
    Sequence Number: 1    (relative sequence number)
    Sequence Number (raw): 399429055
    [Next Sequence Number: 22    (relative sequence number)]
    Acknowledgment Number: 34    (relative ack number)
    Acknowledgment number (raw): 203388201
    0101 .... = Header Length: 20 bytes (5)
    Flags: 0x018 (PSH, ACK)
    Window: 4095
    [Calculated window size: 262080]
    [Window size scaling factor: 64]
    Checksum: 0x629b [unverified]
    [Checksum Status: Unverified]
    Urgent Pointer: 0
    [Timestamps]
    [SEQ/ACK analysis]
    TCP payload (21 bytes)
SSH Protocol
    Protocol: SSH-2.0-OpenSSH_9.7
    [Direction: server-to-client]
 

---

 **🖧 Frame 4462: SSH Key Exchange Initialization**
- **Length:** **1486 bytes**  
- **Source:** `192.168.1.185 (Dell)` → **Destination:** `192.168.1.148 (Mac)`  
- **Protocol:** SSH  
- **Details:**
  - This is a **Key Exchange Initialization (KEXINIT) packet**.
  - The client is sending a list of **supported encryption algorithms** to the server.
  - **Why it matters:** Marks the transition from unencrypted negotiation to **encrypted communication**.
  Frame 4462: 1486 bytes on wire (11888 bits), 1486 bytes captured (11888 bits) on interface \Device\NPF_{EAF66F3F-8F00-47F0-827E-72FB128923A3}, id 0
Ethernet II, Src: e0:ad:47:20:d9:0a (e0:ad:47:20:d9:0a), Dst: Apple_db:fd:32 (74:a6:cd:db:fd:32)
Internet Protocol Version 4, Src: 192.168.1.185, Dst: 192.168.1.148
Transmission Control Protocol, Src Port: 35050, Dst Port: 22, Seq: 34, Ack: 22, Len: 1432
    Source Port: 35050
    Destination Port: 22
    [Stream index: 34]
    [Stream Packet Number: 8]
    [Conversation completeness: Complete, WITH_DATA (47)]
    [TCP Segment Len: 1432]
    Sequence Number: 34    (relative sequence number)
    Sequence Number (raw): 203388201
    [Next Sequence Number: 1466    (relative sequence number)]
    Acknowledgment Number: 22    (relative ack number)
    Acknowledgment number (raw): 399429076
    0101 .... = Header Length: 20 bytes (5)
    Flags: 0x018 (PSH, ACK)
    Window: 256
    [Calculated window size: 65536]
    [Window size scaling factor: 256]
    Checksum: 0xce3b [unverified]
    [Checksum Status: Unverified]
    Urgent Pointer: 0
    [Timestamps]
    [SEQ/ACK analysis]
    TCP payload (1432 bytes)
SSH Protocol
    SSH Version 2 (encryption:chacha20-poly1305@openssh.com mac:<implicit> compression:none)
        Packet Length: 1428
        Padding Length: 10
        Key Exchange (method:curve25519-sha256)
        Padding String: 00000000000000000000
        [Sequence number: 0]
    [Direction: client-to-server]
  

---

 **📌 Step 2: Confirm When Encryption Fully Starts**
Now that we've identified the **handshake and key exchange**, we need to **confirm when encryption is fully enabled**.  

 **🔍 Next Task: Locate the "New Keys" Message**
- This is the final step before encryption begins.
- Search Wireshark for the first packet containing **"New Keys"**.
- Note the **frame number** and **packet length**.
Frame 4467: 70 bytes on wire (560 bits), 70 bytes captured (560 bits) on interface \Device\NPF_{EAF66F3F-8F00-47F0-827E-72FB128923A3}, id 0
Ethernet II, Src: e0:ad:47:20:d9:0a (e0:ad:47:20:d9:0a), Dst: Apple_db:fd:32 (74:a6:cd:db:fd:32)
Internet Protocol Version 4, Src: 192.168.1.185, Dst: 192.168.1.148
Transmission Control Protocol, Src Port: 35050, Dst Port: 22, Seq: 1514, Ack: 1634, Len: 16
    Source Port: 35050
    Destination Port: 22
    [Stream index: 34]
    [Stream Packet Number: 13]
    [Conversation completeness: Complete, WITH_DATA (47)]
    [TCP Segment Len: 16]
    Sequence Number: 1514    (relative sequence number)
    Sequence Number (raw): 203389681
    [Next Sequence Number: 1530    (relative sequence number)]
    Acknowledgment Number: 1634    (relative ack number)
    Acknowledgment number (raw): 399430688
    0101 .... = Header Length: 20 bytes (5)
    Flags: 0x018 (PSH, ACK)
    Window: 256
    [Calculated window size: 65536]
    [Window size scaling factor: 256]
    Checksum: 0x23fe [unverified]
    [Checksum Status: Unverified]
    Urgent Pointer: 0
    [Timestamps]
    [SEQ/ACK analysis]
    TCP payload (16 bytes)
SSH Protocol
    SSH Version 2 (encryption:chacha20-poly1305@openssh.com mac:<implicit> compression:none)
        Packet Length: 12
        Padding Length: 10
        Key Exchange (method:curve25519-sha256)
        Padding String: 00000000000000000000
        [Sequence number: 2]
    [Direction: client-to-server]


📌 **After locating the "New Keys" frame, we will analyze the encrypted file transfer process.**


**🔐 Investigating Encryption in SCP File Transfer (Wireshark Analysis)**

**🔹 Overview**
 We aimed to ensure that the transmitted data was fully encrypted and that no plaintext file contents were exposed.

🛠️ Strategy: How We Investigated Encryption
Filtered Wireshark for SSH Traffic
Applied the filter to isolate SSH traffic:
tcp.port == 22
This displayed only packets related to Secure Shell (SSH) communication, including our SCP transfer.
Identified Key Packets Involved in Data Transfer

We searched for larger packet sizes, as file transfer packets typically carry more data.
The following frames were selected for analysis:
Frame 4525 (Length: 682 bytes)
Frame 4533 (Length: 410 bytes)
🔎 Key Packet Analysis: Confirming Encryption
Below are the critical metadata and observations from the selected frames.

**🔹 Frame 4525 (Server → Client)**
Frame 4525: 682 bytes on wire (5456 bits), 682 bytes captured (5456 bits)
Ethernet II, Src: Apple_db:fd:32 (MacBook), Dst: e0:ad:47:20:d9:0a (Dell)
Internet Protocol Version 4, Src: 192.168.1.148, Dst: 192.168.1.185
Transmission Control Protocol, Src Port: 22, Dst Port: 35050, Seq: 1886, Ack: 2030, Len: 628
SSH Protocol
    SSH Version 2 (encryption: chacha20-poly1305@openssh.com mac:<implicit> compression:none)
    [Direction: server-to-client]

**📌 What This Indicates:**
Encryption Algorithm Used: chacha20-poly1305@openssh.com
Message Authentication Code (MAC): Implicitly included
Compression: None (which is expected for SCP)
Data is fully encrypted, as Wireshark does not reveal any plaintext contents.

**🔹 Frame 4533 (Server → Client)**
Frame 4533: 410 bytes on wire (3280 bits), 410 bytes captured (3280 bits)
Ethernet II, Src: Apple_db:fd:32 (MacBook), Dst: e0:ad:47:20:d9:0a (Dell)
Internet Protocol Version 4, Src: 192.168.1.148, Dst: 192.168.1.185
Transmission Control Protocol, Src Port: 22, Dst Port: 35050, Seq: 2630, Ack: 2126, Len: 356
SSH Protocol
    SSH Version 2 (encryption: chacha20-poly1305@openssh.com mac:<implicit> compression:none)
    Packet Length (encrypted): 8b3df59a
    Encrypted Packet […]: 9cc6fa6375d5dd36f5a11e75edb66de2f164e6196befa3d493956b02931e9fa74a52037cbf0759c09e4fad9e31660e78895bea15c3d237ebf804714869389fc8faf989ad3d7acf7632aec94e81a432ea4284807a3fb9060f590db352130ada0e571e231e74a5d58781085f8
    MAC: 1dfe715ae245c02a20755b022ff6247a
    [Direction: server-to-client]

**📌 What This Indicates:**
Encryption & Integrity Check Confirmed:
The entire payload is encrypted (Encrypted Packet […] section).
The MAC (Message Authentication Code) ensures data integrity.
No Plaintext File Contents Found:
The data in the Packet Length (encrypted) and Encrypted Packet fields confirms encryption.

**📌 Conclusion: SCP Transfer is Fully Encrypted**
✅ All observed SCP packets confirm encryption using chacha20-poly1305@openssh.com.
✅ No plaintext file contents were visible in Wireshark, ensuring confidentiality.
✅ MAC authentication was present, preventing tampering.
✅ The SSH session maintained encryption throughout the transfer.
 This confirms that SCP was properly implemented without data leakage.









**🔍 Investigating the Abrupt Secure Shell (SSH) Connection Closure by the Dell**
🔹 Overview
During our SCP file transfer analysis, we expected to see a clear SSH session termination in Wireshark. However, after reviewing the captured packets, we noticed that the expected SSH termination sequence (FIN/ACK or proper teardown) was missing. This raised concerns about whether the connection had properly closed.

**🛠️ Observations & Initial Concerns**
Wireshark Behavior:
We initially filtered for SSH traffic and expected to see session closure packets.
However, no explicit FIN (finish) or RST (reset) packets appeared in the expected range.
This suggested an incomplete or unexpected termination of the session.
Checking Active SSH Sessions on the MacBook:
We ran:
ps aux | grep sshd
Result: No active sshd (SSH Daemon) processes associated with our session were found.
Conclusion: The MacBook was no longer maintaining an active SSH session.

**🔎 Investigation: How Did the Connection Close?**
To determine the cause of the missing termination packets, we expanded our approach:
Checking for FIN/ACK or Reset Packets in Wireshark
We applied the filter:
tcp.flags.fin==1 || tcp.flags.reset==1
This isolated TCP session closure attempts, revealing one key packet:
Frame 4569: TCP RST (Reset) Packet from Dell → MacBook
**Observation: The Dell initiated a TCP Reset (RST) instead of a clean termination.**

Checking for Open SSH Sessions on the Dell
We ran:
powershell
netstat -ano | findstr :22
Result: No active SSH connections were present, but the Dell's SSH service was still listening.

Checking Windows Event Logs for SCP Transfer Events
We ran:
powershell
Get-WinEvent -LogName System | Where-Object { $_.Message -match "scp" }
Result: No specific SCP-related events were logged.

**Conclusion: The Dell Forcefully Closed the Connection**
Instead of gracefully terminating the SSH session, the Dell sent a TCP RST (Reset) packet.

Possible Reasons:
The SCP process completed its task and terminated abruptly.
The SSH session did not perform a full FIN/ACK closure before terminating.
SCP may forcefully close SSH sessions upon completion in certain implementations.

Why Didn’t We See a FIN/ACK Sequence?
The TCP Reset (RST) overrides a normal FIN/ACK termination, immediately cutting the connection.
This explains why we didn’t see a clean SSH session close in Wireshark.

**🔹 Key Takeaways**
✅ We confirmed that the SSH session was closed, despite the missing expected FIN/ACK sequence.
✅ The Dell forcefully terminated the session using a TCP Reset (RST) packet.
✅ This behavior is not uncommon and depends on SCP and SSH client behavior.
✅ We adjusted our project plan to ensure we verify SSH closure beyond just Wireshark.














